In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
import torch.nn as nn

#28,28
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3),  # 16, 26 ,26
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True))

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3),  # 32, 24, 24
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2))  # 32, 12,12     (24-2) /2 +1

        self.dropout = nn.Dropout(p=0.4)  # dropout训练
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3),  # 64,10,10
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True))
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3),  # 128,8,8
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2))  # 128, 4,4

        self.fc = nn.Sequential(
            nn.Linear(128 * 4 * 4, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 10))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.dropout(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
model = CNN()
print(model)

In [ ]:
import torch
import torch.utils.data
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import numpy as np

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        image = self.data.iloc[index, 1:].values.astype(np.uint8).reshape((28, 28, 1))
        label = self.data.iloc[index, 0]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

In [ ]:
transform=transforms.Compose([
                            transforms.ToPILImage(),
                           # Add random transformations to the image.
                           transforms.RandomAffine(
                               degrees=20,
                               translate=(0.1, 0.2),
                               shear=(-10, 20, -10, 20),
                               scale=(0.60, 1.25)),

                           transforms.ToTensor()
                       ])

In [ ]:
batch_size = 128
train_data = MyDataset('/kaggle/input/Kannada-MNIST/train.csv', transform=transform)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_data = MyDataset('/kaggle/input/Kannada-MNIST/test.csv', transform=transform)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
epoch_num = 20
accuracy = []
losses = []
for epoch in range(epoch_num):
    train_correct = 0
    running_loss = 0
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        var_X_batch = Variable(X_batch).float()
        var_y_batch = Variable(y_batch)
        optimizer.zero_grad()
        
        output = model(var_X_batch)
        loss = criterion(output, var_y_batch)
        running_loss += loss.item()
        
        predicted = torch.max(output.data, 1)[1]
        train_correct += (predicted == var_y_batch).sum()
        batch_accuracy = float(train_correct*100) / float(batch_size*(batch_idx+1)) # in all trained data calculate the accracy
        accuracy.append(batch_accuracy)

        if batch_idx % 50 == 0:
            print('Epoch : {} [{}/{}]  Loss:{:.6f}  Accuracy:{:.2f}%'.format(
                epoch,
                batch_idx*len(X_batch), # how much data has been trained
                len(train_loader.dataset),
                loss.item(),
                batch_accuracy))
        loss.backward()
        optimizer.step()
        
    losses.append(running_loss)

In [ ]:
def plot_loss_accuracy(loss, accuracy):
    fid = plt.figure(figsize=(20,5))

    Axes = plt.subplot(121, title="loss")
    Axes.axes.tick_params(which='both',direction='in',top=True, right=True)
    plt.minorticks_on()
    Axes.set_facecolor((0,0,0,0.02))
    # draw the line
    loss_X = np.arange(0,len(loss))
    plt.plot(loss_X, loss, 'k-', linewidth=3, color = 'r', label='loss')
    plt.grid(True,which='major',linewidth=0.5)
    plt.grid(True,which='minor',linewidth=0.1)
    plt.xlabel("epoch number")
    plt.ylabel("loss")
    plt.legend(loc='best',fontsize='x-small')

    Axes = plt.subplot(122, title="accuracy")
    Axes.axes.tick_params(which='both',direction='in',top=True, right=True)
    plt.minorticks_on()
    Axes.set_facecolor((0,0,0,0.02))
    # draw the line
    accuracy_X = np.arange(0,len(accuracy))
    plt.plot(accuracy_X, accuracy, 'k-', linewidth=3, color = 'b', label='accuracy')
    plt.grid(True,which='major',linewidth=0.5)
    plt.grid(True,which='minor',linewidth=0.1)
    plt.xlabel("epoch number")
    plt.ylabel("accuracy")
    plt.legend(loc='best',fontsize='x-small')

In [ ]:
plot_loss_accuracy(losses, accuracy)

In [ ]:
prediction = []
data_submission = pd.read_csv("/kaggle/input/Kannada-MNIST/sample_submission.csv")
for idx, (test_data, y_label) in enumerate(val_loader):
    var_test_data = Variable(test_data).float()
    output = model(var_test_data)
    prediction.append(torch.max(output.data, 1)[1].numpy())

In [ ]:
res = []
for i in prediction:
    for j in i:
        res.append(j)
data_submission['label'] = res
data_submission.to_csv('./submission.csv',index=False)